In [ ]:
## NN classifier (keras.models.Sequential)
## Recurrent neural network (RNN) - e.g. LSTM keras.layers

In [ ]:
## Graph embedding, Protein seq embedding, Structure embedding, RDKit (??)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Path

In [ ]:
## Load Path
get_path = 'drive/MyDrive/Skip-GNN/'
data_path = get_path + 'data/IAV/'

## Import Libaries

In [ ]:
import pandas as pd
import numpy as np

## Pre-processing
import scipy.sparse as sp
from scipy import sparse

## Logistic Regression
from sklearn.linear_model import LogisticRegression

## NN Classifier
## Sequential model: Plain stack of layers where each layer has exactly one input tensor and one output tensor
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras import callbacks ## Early Stopping

from sklearn.utils import shuffle

from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, f1_score

In [ ]:
protein_list = pd.read_csv(data_path + 'protein_list.csv')
# protein_list.columns = ['idx', 'Protein1_ID']
## protein_list

## Utils

### Metrics

In [ ]:
## Add F1, ROC-AUC and PR-AUC

def calculate_metrics(y_label, y_pred, y_pred_f):

    # For binary classification
    TP,FP,TN,FN = 0,0,0,0
    for i in range(len(y_label)):
        if y_label[i] == y_pred[i]:
            if y_label[i] == 1:
                TP = TP + 1
            else:
                TN = TN + 1
        else:
            if y_pred[i] == 1:
                FP = FP + 1
            else:
                FN = FN + 1
    
    accuracy = (TP + TN) / float(TP + TN + FP + FN)
    sensitivity = TP / float(TP + FN)
    specificity = TN / float(TN + FP)
    precision = TP / float(TP + FP)
    F1 = (2 * precision * sensitivity) / (precision + sensitivity)

    ROC_AUC = roc_auc_score(y_label, y_pred_f)
    PR_AUC = average_precision_score(y_label, y_pred_f)

    return [accuracy, sensitivity, specificity, precision, F1, ROC_AUC, PR_AUC]

### Input data

In [ ]:
def create_fold():

  ## hold-out test set
  test_df = pd.read_csv(data_path + 'edges/nn_test_idx.csv')
  pos_test = test_df[test_df['label'] == 1]
  neg_test = test_df[test_df['label'] == 0]

  ## Training and Validation Set
  pos_org = pd.read_csv(data_path + 'edges/nn_pos_idx.csv')
  neg_org = pd.read_csv(data_path + 'edges/nn_neg_idx.csv')

  '''
    Shuffle Data
  '''
  pos = shuffle(pos_org)
  neg = shuffle(neg_org)

  ## Train : Val = 9 : 1
  pos_val = pos.sample(frac = 0.1, replace = False) ## 10% of positive dataset
  pos_train = pos[~pos.index.isin(pos_val.index)]

  neg_val = neg.sample(frac = 0.1, replace = False) ## 10% of negative dataset
  neg_train = neg[~neg.index.isin(neg_val.index)]

  # train_df = pd.concat([pos_train, neg_train], ignore_index=True)
  # val_df = pd.concat([pos_val, neg_val], ignore_index=True)

  print('--Sampled new data--')

  # return train_df, val_df, test_df
  return pos_train, neg_train, pos_val, neg_val, pos_test, neg_test ## train_edges, train_edges_false, val_edges, val_edges_false, test_edges, test_edges_false

In [ ]:
## Read embeddings of all proteins

def read_embedding_matrix(emb_mtd):
  
  protein_list_len = len(protein_list)

  ## Read embedding
  edit_data_path = data_path + 'graph_embeddings/SDNE/'
  # emb = pd.read_csv(edit_data_path + emb_mtd + '.csv', skiprows=1, header = None).sort_values(by = [0]).set_index([0]) 
  emb = pd.read_csv(edit_data_path + emb_mtd + '.txt', sep=' ', skiprows=1, header = None).sort_values(by = [0]).set_index([0]) ## SDNE, GraRep

  ## Convert csv to array
  for i in np.setdiff1d(np.arange(protein_list_len), emb.index.values): ## setdiff1d: 1D array of values in ar1 that are not in ar2
      emb.loc[i] = (np.sum(emb.values, axis = 0)/emb.values.shape[0]) ## manually insert emb for protein indexes with no node2vec embedding

  features = emb.sort_index().values

  print("---Read Embeddings---")
  print(features.shape)
  # print(features)

  return features

### Retrieve embeddings

In [ ]:
## Retrieve embeddings w.r.t X_train, y_train, X_test, y_test
def retrieve_edge_embeddings(emb, posEdges, negEdges):
  X = []
  Y = []

  for i in posEdges.index:
    node_u_emb = emb[posEdges['Protein1_ID'][i]]
    node_v_emb = emb[posEdges['Protein2_ID'][i]]
    feature_vector = np.append(node_u_emb, node_v_emb)
    X.append(feature_vector)
    Y.append(1)
  
  for i in negEdges.index:
    node_u_emb = emb[negEdges['Protein1_ID'][i]]
    node_v_emb = emb[negEdges['Protein2_ID'][i]]
    feature_vector = np.append(node_u_emb, node_v_emb)
    X.append(feature_vector)
    Y.append(0)

  # print("---Generate data---")
  # # print(X) ## list
  # print(len(X)) 
  # # print(Y)
  # print(len(Y))

  return X, Y

## Logistic Regression

In [ ]:
def trainLG(X_train, y_train, X_val, y_val, X_test, y_test):
  
  clf1 = LogisticRegression(max_iter=3000)
  clf1.fit(X_train, y_train)

  y_pred_proba = clf1.predict_proba(X_test)[:, 1]
  y_pred = clf1.predict(X_test)

  auc_roc = roc_auc_score(y_test, y_pred_proba)
  auc_pr = average_precision_score(y_test, y_pred_proba)
  accuracy = accuracy_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)

  acc = calculate_metrics(y_test, y_pred, y_pred_proba)

  return acc

## main

In [ ]:
def train(input_type):

    ## Call create_fold()
    train_edges, train_edges_false, val_edges, val_edges_false, test_edges, test_edges_false = create_fold()

    print(train_edges.shape,train_edges_false.shape)
    print(val_edges.shape,val_edges_false.shape)
    print(test_edges.shape,test_edges_false.shape)

    ## Retrieve embeddings of all proteins 
    emb = read_embedding_matrix(input_type) ## embedding method

    ## Retrieve embeddings of respective nodes 
    X_train,Y_train = retrieve_edge_embeddings(emb, train_edges, train_edges_false)
    X_val, Y_val = retrieve_edge_embeddings(emb, val_edges, val_edges_false)
    X_test,Y_test = retrieve_edge_embeddings(emb, test_edges, test_edges_false)
  
    ## Final softmax classifier
    # acc = train_nn(X_train,Y_train,X_val,Y_val,X_test,Y_test)
    acc = trainLG(X_train, Y_train, X_val, Y_val, X_test, Y_test)

    return acc

In [ ]:
## TEST RUN
# acc = train(input_type='node2vec')
# print(acc)

In [ ]:
## 5-fold Cross Validation
eval_metrics = []

for i in range(0, 5):

    print('Iteration(train): ', (i+1))
    acc = train(input_type='a0_b0') ## Change input type

    print('---Link Prediction Performance---')
    print(acc)
   
    eval_metrics.append(acc)
    # print(eval_metrics)

Iteration(train):  1
--Sampled new data--
(3422, 3) (3422, 3)
(380, 3) (380, 3)
(422, 3) (422, 3)
---Read Embeddings---
(15685, 128)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


---Link Prediction Performance---
[0.9786729857819905, 0.9834123222748815, 0.9739336492890995, 0.9741784037558685, 0.9787735849056605, 0.997748253633117, 0.997697972768367]
Iteration(train):  2
--Sampled new data--
(3422, 3) (3422, 3)
(380, 3) (380, 3)
(422, 3) (422, 3)
---Read Embeddings---
(15685, 128)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


---Link Prediction Performance---
[0.9774881516587678, 0.981042654028436, 0.9739336492890995, 0.9741176470588235, 0.9775678866587957, 0.9975236405291885, 0.9974352657727892]
Iteration(train):  3
--Sampled new data--
(3422, 3) (3422, 3)
(380, 3) (380, 3)
(422, 3) (422, 3)
---Read Embeddings---
(15685, 128)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


---Link Prediction Performance---
[0.9786729857819905, 0.9834123222748815, 0.9739336492890995, 0.9741784037558685, 0.9787735849056605, 0.9975573324947777, 0.9974794944396794]
Iteration(train):  4
--Sampled new data--
(3422, 3) (3422, 3)
(380, 3) (380, 3)
(422, 3) (422, 3)
---Read Embeddings---
(15685, 128)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


---Link Prediction Performance---
[0.9786729857819905, 0.9834123222748815, 0.9739336492890995, 0.9741784037558685, 0.9787735849056605, 0.9978437142022866, 0.99779684106588]
Iteration(train):  5
--Sampled new data--
(3422, 3) (3422, 3)
(380, 3) (380, 3)
(422, 3) (422, 3)
---Read Embeddings---
(15685, 128)
---Link Prediction Performance---
[0.9774881516587678, 0.981042654028436, 0.9739336492890995, 0.9741176470588235, 0.9775678866587957, 0.9976752543743401, 0.9976074775543173]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


## Evaluation

In [ ]:
## Append results here
eval_arr = eval_metrics

In [ ]:
mean = np.array(eval_arr).mean(axis=0) # Take the mean of each column
mean = np.round(mean, 4)
print('Mean: ' + str(mean)[1:-1])

max = np.array(eval_arr).max(axis=0)
max = np.round(max, 4)
print('Max: ' + str(max)[1:-1])

min = np.array(eval_arr).min(axis=0)
min = np.round(min, 4)
print('Min: ' + str(min)[1:-1])

Mean: 0.9782 0.9825 0.9739 0.9742 0.9783 0.9977 0.9976
Max: 0.9787 0.9834 0.9739 0.9742 0.9788 0.9978 0.9978
Min: 0.9775 0.981  0.9739 0.9741 0.9776 0.9975 0.9974


In [ ]:
## CHANGE METHOD HERE
mtd_name = 'SDNE-a0_b0'

In [ ]:
with open(get_path + 'NN_results.txt', "a") as f:
    f.write(mtd_name + ': ' + str(mean) + '\n')

## Additional codes

In [ ]:
### Map edges to index in protein list

# pos_org = pd.read_csv(data_path + 'edges/train_val_pos_tune_hyperparams.csv')
# pos_org_m1 = pd.merge(pos_org, protein_list, on=['Protein1_ID'])
# pos_org_m1 = pd.merge(pos_org_m1, protein_list, left_on=['Protein2_ID'], right_on=['Protein1_ID'])
# # pos_org_m1

# pos_org_f = pos_org_m1[['idx_x', 'idx_y', 'label']]
# pos_org_f
# # pos_org_f.to_csv(data_path + 'edges/nn_pos_idx.csv', index=False, header=['Protein1_ID', 'Protein2_ID', 'label'])

# neg_org = pd.read_csv(data_path + 'edges/train_val_neg_tune_hyperparams.csv')
# neg_org_m1 = pd.merge(neg_org, protein_list, on=['Protein1_ID'])
# neg_org_m1 = pd.merge(neg_org_m1, protein_list, left_on=['Protein2_ID'], right_on=['Protein1_ID'])

# neg_org_f = neg_org_m1[['idx_x', 'idx_y', 'label']]
# neg_org_f
# # neg_org_f.to_csv(data_path + 'edges/nn_neg_idx.csv', index=False, header=['Protein1_ID', 'Protein2_ID', 'label'])

## hold-out test set
# test_df = pd.read_csv(data_path + 'edges/test_holdout_tune_hyperparams.csv')

In [ ]:
## trainNN
# def generate_data(emb, posEdges, negEdges):
    
#     ## Stack embeddings of two proteins together
#     posNum = posEdges.shape[0]
#     negNum = negEdges.shape[0]

#     X = np.empty((posNum+negNum,2*emb.shape[1]))
#     k = 0

#     # for x in posEdges.index:
#     #     X[k] = np.hstack((emb[x[0]],emb[x[1]]))
#     #     k = k + 1
#     # for x in negEdges.index:
#     #     X[k] = np.hstack((emb[x[0]],emb[x[1]]))
#     #     k = k + 1 

#     for i in posEdges.index:
#        X[k] = np.hstack((emb[posEdges['Protein1_ID'][i]], emb[posEdges['Protein2_ID'][i]]))
#        k = k + 1
    
#     for i in negEdges.index:
#       X[k] = np.hstack((emb[negEdges['Protein1_ID'][i]], emb[negEdges['Protein2_ID'][i]]))
#       k = k + 1

#     Y_pos = np.full((posNum,2),[0,1])
#     Y_neg = np.full((negNum,2),[1,0])
#     Y = np.vstack((Y_pos,Y_neg))

#     print("---Generate data---")
#     print(X)
#     print(X.shape) ## (2D array)
#     print(Y) ## Class label (2D array)
#     print(Y.shape)

#     return X,Y

### trainNN Model

In [ ]:
# def train_nn(X_train, Y_train, X_val, Y_val, X_test, Y_test): 

#     model = Sequential()
#     model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
#     model.add(Dropout(0.5))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(32, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(2,activation='softmax'))

#     model.compile(loss='categorical_crossentropy',
#                     optimizer='adam',
#                     metrics=['accuracy'])

#     ## Early stopping on validation dataset (10% of overall dataset)
#     earlystopping = callbacks.EarlyStopping(monitor='val_loss', mode="min", patience=5, restore_best_weights=True)
#     model.fit(X_train, Y_train, epochs=200, batch_size=128, validation_data=(X_val, Y_val), callbacks=[earlystopping])

#     ## Save model to make future predictions
#     model.save(get_path + 'NN_model/model.h5')
#     print("---Saved model to disk---")

#     ### Using Hold-out Test Set (X_test and Y_test)
#     # y_prob = model.predict(X_test) 
#     print(y_prob)
#     y_classes = y_prob.argmax(axis=-1) ## Binary prediction (y_pred)

#     y_pred_float = model.predict_proba(X_test) ## Float prediction
#     y_pred_f = y_pred_float[:,1] ## Probability of belonging to class label '1' 
#     print(y_pred_float) 
#     print(y_pred_f)

#     y_true = Y_test[:,1]

#     acc = calculate_metrics(y_true, y_classes, y_pred_f) ## calculate_metrics(y_label, y_pred, y_pred_f)

#     return acc